In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset

#import function for state geo data boundaries
import state_locations

#functions for datetime management
import datetime
from cftime import num2pydate

In [2]:
#### Loads MERRA netcdf file and important variables ####
# Note: operates over one day of data based on the daily nature of each MERRA file # 
def get_data_from_netcdf(filename):
    data = Dataset(filename)
    
    ### Extracts location variables and other important variables from "data" ###
    lons = data.variables['lon'][:]
    lats = data.variables['lat'][:]
    hurs = data.variables['hurs'][:,:,:]
    
    # uses numbers and atributes from the CF time library to get the actual date
    times = data.variables['time']
    # this function just creates an object with python data objects
    time = num2pydate(times,units=times.units,calendar=times.calendar)
    
    # cycle to get every date from the object from the previous function
    for i in range(len(time)):
        time[i] = time[i].strftime('%m/%d/%Y')
    
    # print the variables saved on the dataset
    #print(data.variables)
  
    
    # Consolidates time, location, and variables of interest into one dataframe ###
    names = ['t','y','x']
    index = pd.MultiIndex.from_product([time,lats,lons], names=names)
    df = pd.DataFrame({'hurs': hurs.flatten()}, index=index)['hurs']
    df.index.names = ['time', 'lats', 'lons']
    df = df.reset_index(level=['time', 'lats', 'lons'])
    #print(df)
    
    # get the data for the state of interest
    W, E, S, N = state_locations.get_state_loc('IN')
    
    # create a subset from the bigger dataframe
    INDIANA= df[((df["lats"]> W)&(df["lats"]< E))&((df["lons"]> S)&(df["lons"]< N))]
    
    #print(INDIANA)
    
    # output to a csv file
    INDIANA.to_csv('indiana.csv',index=False)

    
    
    


In [3]:
get_data_from_netcdf("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")
#organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")
#organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")
#organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")

                 time   lats   lons       hurs
251715     01/01/2006 -84.75  37.75  75.384926
251716     01/01/2006 -84.75  38.25  75.050568
251717     01/01/2006 -84.75  38.75  72.961006
251718     01/01/2006 -84.75  39.25  73.098534
251719     01/01/2006 -84.75  39.75  73.249413
...               ...    ...    ...        ...
473296039  12/31/2010 -87.75  39.75  90.803444
473296040  12/31/2010 -87.75  40.25  90.904510
473296041  12/31/2010 -87.75  40.75  91.029114
473296042  12/31/2010 -87.75  41.25  91.027733
473296043  12/31/2010 -87.75  41.75  91.056358

[115038 rows x 4 columns]
